# Proyecto Webscraping + DataViz
## Objetivos
El objetivo es construir un proyecto propio a partir del curso de Domestika **Introducción a la visualización de datos**.

Skills a desarrollar:
- Webscraping desde python con la libreria **beautifulsoup** en vez de con una extensión de google (imdb.com)
- Crear un dataframe (df) dinámico dependiente del imput del usuario (elección de series de la que extraer los datos)
- EDA dinámico: según los datos extraidos por el usuario
- Transformación a formato excel del df
- Construcción de un dashboard dinámico dependiente del df del usuatrio
- **Extra**: convertirlo en una app con streamlit

## Parte 1: Webscraping
Queremos contruir una funcion-clase que sea capaz de acceder a la página de imdb.com, a la serie seleccionado por el usuario mediante un imput manual. Tiene que ser capaz de:
- Acceder a la pagina de la serie correctamente
- Acceder al apartado de capítulos
- Extraer los datos relevantes de los capitulos:
    - Numero de temporada
    - Numero de episodio
    - Fecha de estreno
    - Nombre del episodio
    - Puntuación 
    - Número de veces puntuado
- Incluir cada dato en la columna del df correcta con el formato adecuado
- En caso de tener mas de una temporada, realizar el mismo proceso para cada una de las temporadas

### Página IMBD
**Dificultades:** 
- Las urls de imdb.com no son muy intuitivas por lo que puede ser un problema a la hora de navegar entre ellas.
    - Ejemplo **From** temporada 1: https://www.imdb.com/es-es/title/tt9813792/episodes/?season=1&ref_=ttep
    - Ejemplo **Arcane** temporada 2: https://www.imdb.com/es-es/title/tt11126994/episodes/?season=2&ref_=ttep

La clave va a ser encontrar el identificador de la serie **tt+numeros**


### Beautifulsoup
Pruebas iniciales

In [6]:
# Instalamos las librerías request y beautifulsoup
import sys
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install beautifulsoup4

  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.

[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.3.1 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# Importamos las librerias
import requests
from bs4 import BeautifulSoup

In [ ]:
# Probamos la llamada a la url con la serie Arcane temporada 1 (formato text)
arcane_t1 = requests.get("https://www.imdb.com/es-es/title/tt11126994/episodes/?season=1&ref_=ttep").text

In [18]:
# Mostramos lo que hay en nuestra nueva variable
print(arcane_t1)

<Response [403]>


In [22]:
sopa = BeautifulSoup(arcane_t1,'html.parser')

In [23]:
sopa

<html>
<head><title>403 Forbidden</title></head>
<body>
<center><h1>403 Forbidden</h1></center>
</body>
</html>

La página imdb no me deja acceder a la totalidad de sus datos. Tratamos de "engañar" a la página usando la opción de "headers"

In [24]:
# URL del listado de episodios
url = "https://www.imdb.com/es-es/title/tt11126994/episodes/?season=1&ref_=ttep"

# Simulamos ser un navegador real
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
                  "AppleWebKit/537.36 (KHTML, like Gecko) "
                  "Chrome/112.0.0.0 Safari/537.36",
    "Accept-Language": "es-ES,es;q=0.9"
}

# Petición
response = requests.get(url, headers=headers)

# Verificamos que haya ido bien
print(response.status_code)  # Debería dar 200

# Parseamos si todo está OK
if response.status_code == 200:
    soup = BeautifulSoup(response.content, "html.parser")
    print(soup.prettify()[:1000])  # Solo mostramos un trozo para que no se sature
else:
    print("Algo ha fallado")

200
<!DOCTYPE html>
<html lang="es-ES" xmlns:fb="http://www.facebook.com/2008/fbml" xmlns:og="http://opengraphprotocol.org/schema/">
 <head>
  <meta charset="utf-8"/>
  <meta content="width=device-width" name="viewport"/>
  <script>
   if(typeof uet === 'function'){ uet('bb', 'LoadTitle', {wb: 1}); }
  </script>
  <script>
   window.addEventListener('load', (event) => {
        if (typeof window.csa !== 'undefined' && typeof window.csa === 'function') {
            var csaLatencyPlugin = window.csa('Content', {
                element: {
                    slotId: 'LoadTitle',
                    type: 'service-call'
                }
            });
            csaLatencyPlugin('mark', 'clickToBodyBegin', 1745057569488);
        }
    })
  </script>
  <title>
   Arcane (Serie de TV 2021–2024) - Lista de episodios - IMDb
  </title>
  <meta content="Arcane (Serie de TV 2021–2024) - Películas, televisión, celebridades y más..." data-id="main" name="description"/>
  <meta content="Episod